In [1]:
END_YEAR = 2024

In [2]:
teamRunCounts = {}
gameData = {}

In [3]:
import sqlite3
from tqdm import tqdm

Empty out so don't have to deal with duplicates

In [4]:
db = sqlite3.connect("BaseballStats.db")
db.rollback()
cursor = db.cursor()
cursor.execute("DELETE FROM Park_Factors")
cursor.execute("DELETE FROM Park_ScoringData")
cursor.execute("DELETE FROM League_Factors")
cursor.execute("DELETE FROM Level_Factors")
cursor.execute("DELETE FROM Player_Hitter_GameLog WHERE Level>?", (17,))
cursor.execute("DELETE FROM Player_Pitcher_GameLog WHERE Level>?", (17,))
db.commit()

Handle dictionary adds

In [5]:
def CheckForKeysAndAddIfNecessary(t, l, year, level, teamRunCounts):
    if not l in teamRunCounts.keys():
        teamRunCounts[l] = {}
    if not t in teamRunCounts[l].keys():
        teamRunCounts[l][t] = {}
    if not year in teamRunCounts[l][t].keys():
        teamRunCounts[l][t][year] = {"home": {"outs":0,"pa":0,"runs":0,"hrs":0}, "away": {"outs":0,"pa":0,"runs":0,"hrs":0}, "level": level, "league": l, "team":t}

In [6]:
def UpdateGameData(gameId, pa, outs, r, hr, homeTeamId, teamId, leagueId, year, level):
    global gameData
    if not gameId in gameData.keys():
        gameData[gameId] = {"homeTeamId":homeTeamId, "awayTeamId":0, "r":0,"hr":0,"pa":0,"outs":0, "league":leagueId, "level":level, "year":year}
    
    gd = gameData[gameId]
    if homeTeamId != teamId:
        gd["awayTeamId"] = teamId
        
    gd["pa"] += pa
    gd["outs"] += outs
    gd["hr"] += hr
    gd["r"] += r

Create dictionary of all games to track home/away runs

In [7]:
cursor = db.cursor()
gameLogs = cursor.execute("SELECT gameId, year, month, level, homeTeamId, TeamId, LeagueId, battersFaced, outs, r, hr FROM Player_Pitcher_GameLog").fetchall()
for gameId, year, month, level, homeTeamId, teamId, leagueId, battersFaced, outs, r, hr in tqdm(gameLogs):
    UpdateGameData(gameId, battersFaced, outs, r, hr, homeTeamId, teamId, leagueId, year, level)

100%|██████████| 1906167/1906167 [00:01<00:00, 1397494.29it/s]


Track how many runs each team gives up in home/away games

In [8]:
db.rollback()
cursor = db.cursor()
for val in gameData.values():
    # Get values
    leagueId = val["league"]
    homeTeamId = val["homeTeamId"]
    awayTeamId = val["awayTeamId"]
    runs = val["r"]
    hrs = val["hr"]
    pa = val["pa"]
    outs = val["outs"]
    year = val["year"]
    level = val["level"]
    
    if awayTeamId == 0:
        continue
    
    # Update for each time
    CheckForKeysAndAddIfNecessary(homeTeamId, leagueId, year, level, teamRunCounts)
    teamRunCounts[leagueId][homeTeamId][year]["home"]["outs"] += outs
    teamRunCounts[leagueId][homeTeamId][year]["home"]["pa"] += pa
    teamRunCounts[leagueId][homeTeamId][year]["home"]["runs"] += runs
    teamRunCounts[leagueId][homeTeamId][year]["home"]["hrs"] += hrs
    
    CheckForKeysAndAddIfNecessary(awayTeamId, leagueId, year, level, teamRunCounts)
    teamRunCounts[leagueId][awayTeamId][year]["away"]["outs"] += outs
    teamRunCounts[leagueId][awayTeamId][year]["away"]["pa"] += pa
    teamRunCounts[leagueId][awayTeamId][year]["away"]["runs"] += runs
    teamRunCounts[leagueId][awayTeamId][year]["away"]["hrs"] += hrs

Insert home/away scores for each teams at home/away

In [9]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")
insertionData = []
for leagueId, teamData in tqdm(teamRunCounts.items()):
    for teamid, yearData in teamData.items():
        for year, parkData in yearData.items():
            valueExists = cursor.execute(f"SELECT COUNT(*) FROM Park_ScoringData WHERE TeamId='{teamid}' AND Year='{year}'").fetchone()[0] > 0
            if valueExists:
                continue
                
            homePA = parkData["home"]["pa"]
            homeOuts = parkData["home"]["outs"]
            homeRuns = parkData["home"]["runs"]
            homeHRs = parkData["home"]["hrs"]
            awayPA = parkData["away"]["pa"]
            awayOuts = parkData["away"]["outs"]
            awayRuns = parkData["away"]["runs"]
            awayHRs = parkData["away"]["hrs"]
            levelId = parkData["level"]
            insertionData.append([teamid, year, levelId, leagueId, homePA, homeOuts, homeRuns, homeHRs, awayPA, awayOuts, awayRuns, awayHRs])
cursor.executemany("INSERT INTO Park_ScoringData('TeamId','Year','LevelId','LeagueId','HomePA','HomeOuts','HomeRuns','HomeHRs','AwayPA','AwayOuts','AwayRuns','AwayHRs') VALUES (?,?,?,?,?,?,?,?,?,?,?,?)", insertionData)
cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 20/20 [00:00<00:00, 460.06it/s]


Calculate Park Factors

In [10]:
ROLLING_PERIOD = 3
INNING_CUTOFF = 100

db = sqlite3.connect("BaseballStats.db")
cursor = db.cursor()
scoringData = cursor.execute("SELECT DISTINCT TeamId, Year FROM Park_ScoringData").fetchall()

for team, year in tqdm(scoringData):
    # Check if data already exists
    if cursor.execute(f"SELECT COUNT(*) FROM Park_Factors WHERE year='{year}' AND TeamId='{team}'").fetchone()[0] > 0:
        #print("Continuing")
        continue
    
    parkRunData = cursor.execute(f"SELECT * FROM Park_ScoringData WHERE TeamId='{team}' AND year > '{year - ROLLING_PERIOD}' AND year <= '{year}' ORDER BY Year DESC, HomeOuts DESC").fetchall()
    # Check if data exists
    if len(parkRunData) == 0:
        continue
    
    # Make sure there is some data from the current year
    if parkRunData[0][1] != year:
        continue
    
    # Get current league from first entry
    leagueId = parkRunData[0][2]
    levelId = parkRunData[0][3]
    
    awayOuts = 0
    awayPa = 0
    awayRuns = 0
    awayHRs = 0
    homeOuts = 0
    homePa = 0
    homeRuns = 0
    homeHRs = 0
    for _,_,_,_,hpa, ho,hr,hhr,apa,ao,ar,ahr in parkRunData:
        awayOuts += ao
        awayPa += apa
        awayRuns += ar
        awayHRs += ahr
        homeOuts += ho
        homePa += hpa
        homeRuns += hr
        homeHRs += hhr
        
    # Ensure enough data to actually calculate park factors
    if awayOuts * 3 < INNING_CUTOFF or homeOuts * 3 < INNING_CUTOFF:
        continue
    
    runFactor = (homeRuns / homeOuts) / (awayRuns / awayOuts)
    hrFactor = (homeHRs / homePA) / (awayHRs / awayPa)
    params = [(team, leagueId, levelId, year, runFactor, hrFactor)]
    cursor.executemany("INSERT INTO Park_Factors('TeamId','LeagueId','LevelId','Year','RunFactor','HRFactor') VALUES(?,?,?,?,?,?)", params)
    db.commit()

100%|██████████| 4808/4808 [00:05<00:00, 911.00it/s]


Calculate Level Factors

In [11]:
db.rollback()
cursor = db.cursor()

for year in cursor.execute("SELECT DISTINCT Year FROM Park_Factors").fetchall():
    year = year[0]
    data = []
    totalOuts = 0
    totalPa = 0
    totalRuns = 0
    totalHrs = 0
    for level in cursor.execute(f"SELECT DISTINCT LevelId FROM Park_Factors WHERE Year='{year}'").fetchall():
        level = level[0]
        # Covid check
        if year == 2020 and level != 1:
            continue
        if year > 2020 and level == 15:
            continue
        pa, outs, runs, hrs = cursor.execute(f"SELECT SUM(HomePa), SUM(HomeOuts), SUM(HomeRuns), SUM(HomeHRs) FROM Park_ScoringData WHERE Year='{year}' AND LevelId='{level}'").fetchone()
        
        data.append((level, runs/outs, hrs/pa))
        totalOuts += outs
        totalPa += pa
        totalRuns += runs
        totalHrs += hrs
        
    baseRunFactor = totalRuns / totalOuts
    baseHRFactor = totalHrs / totalPa
    cursor.execute("BEGIN TRANSACTION")
    for d in data:
        if cursor.execute(f"SELECT COUNT(*) FROM Level_Factors WHERE LevelId='{d[0]}' AND Year='{year}'").fetchone()[0] == 0:
            cursor.execute("INSERT INTO Level_Factors('LevelId','Year','RunFactor','HRFactor') VALUES(?,?,?,?)", [d[0], year, d[1] / baseRunFactor, d[2] / baseHRFactor])
    cursor.execute("END TRANSACTION")
    db.commit()
    cursor = db.cursor()

Calculate League Factors

In [12]:
leagues = cursor.execute("SELECT DISTINCT LeagueId FROM Park_Factors").fetchall()
db.rollback()
cursor = db.cursor()

for year in range(2005, END_YEAR + 1):
    # Check if data already exists for year
    if cursor.execute(f"SELECT COUNT(*) FROM League_Factors WHERE Year='{year}'").fetchone()[0] > 0:
        continue
    
    # Get data for each league
    yearlyLeagueData = []
    for league in leagues:
        league = league[0]
        
        data = cursor.execute(f"SELECT HomePa, HomeOuts, HomeRuns, HomeHRs, LevelId FROM Park_ScoringData WHERE LeagueId='{league}' AND Year='{year}'").fetchall()
        homePa = 0
        homeOuts = 0
        homeRuns = 0
        homeHRs = 0
        for pa, outs, runs, hrs, _ in data:
            homePa += pa
            homeOuts += outs
            homeRuns += runs
            homeHRs += hrs
            
        # if league == 134:
        #     print(f"Year={year} HomeInnings={homeInnings}")
            
        if len(data) > 0:
            yearlyLeagueData.append((league, homePa, homeOuts, homeRuns, homeHRs, data[0][4]))
        
    # Get Average of all leagues
    totalOuts = 0
    totalPa = 0
    totalRuns = 0
    totalHr = 0
    for _, pa, outs, r, hr, _ in yearlyLeagueData:
        totalPa += pa
        totalOuts += outs
        totalRuns += r
        totalHr += hr
        
    # Normalize each league to the average
    cursor.execute("BEGIN TRANSACTION")
    dbData = []
    for leagueId, leaguePa, leagueOuts, leagueRuns, leagueHRs, levelId in yearlyLeagueData:
        levelRunFactor, levelHRFactor = cursor.execute(f"SELECT RunFactor, HRFactor FROM Level_Factors WHERE LevelId='{levelId}' AND Year='{year}'").fetchone()
        if leagueOuts * 3 > INNING_CUTOFF:
            dbData.append((leagueId, year, (leagueRuns / leagueOuts)/(totalRuns / totalOuts)/levelRunFactor, (leagueHRs / leaguePa)/(totalHr / totalPa)/levelHRFactor))
        else:
            dbData.append((leagueId, year, 1, 1))
    
    cursor.executemany("INSERT INTO League_Factors('LeagueId','Year','RunFactor','HRFactor') VALUES(?,?,?,?)", dbData)
    cursor.execute("END TRANSACTION")
    db.commit()